# Walking Kodee

This notebook demonstrates how to create an animated mascot (Kodee) that walks across the IntelliJ IDEA interface using the IntelliJ Platform integration. The notebook shows how to:

- Load and display animated GIF images in the IDE
- Create custom Swing components for animations
- Use the IntelliJ Platform API to interact with the IDE frame
- Implement proper resource management with disposables
- Handle EDT (Event Dispatch Thread) operations safely

The main feature creates an animated Kodee character that continuously walks across the bottom of the IDE window, with the animation cycling through GIF frames and repositioning the character when it reaches the edge of the screen.


In [ ]:
%use intellij-platform

In [ ]:
import com.intellij.openapi.application.runInEdt
import com.intellij.openapi.util.Disposer
import com.intellij.openapi.wm.WindowManager
import java.awt.Dimension
import java.awt.Graphics
import java.net.URL
import javax.imageio.ImageIO
import javax.imageio.ImageReader
import javax.swing.JLabel
import kotlin.concurrent.thread

private inline fun <T> ImageReader.use(block: (ImageReader) -> T): T = try {
    block(this)
} finally {
    dispose()
}

class AnimatedGifLabel(private val gifUrl: URL, private val parentWidth: Int) : JLabel() {
    private var currentFrame = 0
    private var xPosition = 0
    private val frames by lazy {
        gifUrl.openStream().use { stream ->
            ImageIO.getImageReadersByFormatName("gif").next().use { reader ->
                reader.input = ImageIO.createImageInputStream(stream)
                (0 until reader.getNumImages(true)).map { reader.read(it) }
            }
        }
    }

    init {
        frames.firstOrNull()?.apply {
            preferredSize = Dimension(width / 2, height / 2)
        }
    }

    override fun paintComponent(g: Graphics) {
        g.drawImage(frames[currentFrame], 0, 0, width, height, null)
    }

    fun startAnimation() = thread {
        generateSequence(0) { (it + 1) % frames.size }.forEach { frameIndex ->
            currentFrame = frameIndex
            repaint()
            Thread.sleep(40)
            xPosition += 3

            if (xPosition > parentWidth) {
                xPosition = -width  // Start from left edge again
            }

            setLocation(xPosition, y)
        }
    }
}

runInEdt {
    val project = currentProject() ?: error("Project not found")
    val frame = WindowManager.getInstance().getFrame(project) ?: error("IDE frame not found")
    val kodee = javaClass.getResource("Kodee.gif") ?: error("Kodee.gif not found")

    val label = AnimatedGifLabel(kodee, frame.width).apply {
        setBounds(
            0,
            frame.height - preferredSize.height - 32,
            preferredSize.width,
            preferredSize.height,
        )
        startAnimation()
    }

    val layeredPane = frame.layeredPane.apply { add(label) }

    Disposer.register(notebookDisposable) {
        layeredPane.apply {
            remove(label)
            revalidate()
            repaint()
        }
    }
}

Loading `Kodee.gif` from resources is available only in IntelliJ IDEA 2025.2+.

For a backward compatibility, you may use:
```kotlin
val kodee = notebook.workingDir.resolve("Kodee.gif").toUri().toURL()
```